In [272]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
import os

In [240]:
au = 1.496e11
G = 6.67e-11

In [123]:
class particle:
    def __init__(self, pos, velocity=0., mass=1.):
        self.pos = float(pos)
        self.velocity = float(velocity)
        self.mass = float(mass)
    
    def update_grav(self, accel):
        self.velocity += accel
        
    def move(self):
        self.pos += self.velocity

In [53]:
def get_colours(palette, n):
    clrs = []
    cmap = matplotlib.cm.get_cmap(palette, n)
    for i in range(cmap.N):
        rgb = cmap(i)[:3]
        clrs.append(matplotlib.colors.rgb2hex(rgb))
    return clrs

In [81]:

def grav(part_a, part_b):
    dist = part_b.pos - part_a.pos
    if dist != 0:
        on_a = G * part_b.mass / dist
    else:
        on_a = 0
    part_a.update_grav(on_a)
    part_b.update_grav(-on_a)

In [82]:
def step(parts):
    for aidx, aval in enumerate(parts):
        for bidx, bval in enumerate(parts[aidx:]):
            grav(aval, bval)
    for i in parts:
        i.move()

In [241]:
def plot_parts(parts, save=None):
    fig = plt.figure(figsize=(10, 10))
    plt.xlim(-5*au, 5*au)
    plt.ylim(-5*au, 5*au)
    plt.scatter(*zip(*[(i.pos, i.velocity) for i in parts]), c=clrs)
    if save:
        plt.savefig(save)
    plt.close()

In [294]:
def integrate(parts, steps, savedir=None, start=0):
    for i in np.arange(start, start+steps):
        if savedir:
            pth = f'{savedir}/fig_{i+1:04}.png'
        plot_parts(parts, save=pth)
        step(parts)
    os.system(f'ffmpeg -framerate 30 -pattern_type glob -i \'{savedir}/fig_*.png\' -c:v libx264 -pix_fmt yuv420p {savedir}/out.mp4')

In [298]:
n_bodies = 5
init_pos = np.linspace(-(n_bodies // 2)*au, (n_bodies // 2)*au, n_bodies)
clrs = get_colours('autumn', n_bodies)

In [299]:
init_parts = []

for pos, clr in zip(init_pos, init_clrs):
    part = particle(pos, mass=5e28)
    init_parts.append(part)

In [300]:
integrate(init_parts, 100, 'grav2', start=0)